<img align='center' style='max-width: 1000px' src='banner.png'>

<img align='right' style='max-width: 200px; height: auto' src='hsg_logo.png'>

##  Lab 08 - Transformers for Summarization (FLAN-T5)

GSERM Summer School 2023, Deep Learning: Fundamentals and Applications, University of St. Gallen

The lab environment of the "Deep Learning: Fundamentals and Applications" GSERM course at the University of St. Gallen (HSG) is based on Jupyter Notebooks (https://jupyter.org), which allow to perform a variety of statistical evaluations and data analyses.

_This worksheet is largely based on HuggingFace's Tech Lead Philipp Schmid's [excellent tutorial](https://www.philschmid.de/fine-tune-flan-t5)._

# Fine-tuning FLAN-T5 for chat and dialogue summarization

In this lab, we will fine-tune [google/flan-t5-small](https://huggingface.co/google/flan-t5-small), a special transformer foundation model, for chat and dialogue summarization using Hugging Face Transformers. 
While it is based on the T5 setup (even in number of parameters), FLAN-T5 has been fine-tuned on more than 1000 additional tasks covering also more languages. 

In this example we will use the [samsum](https://huggingface.co/datasets/samsum) dataset which is a collection of about 16k messenger-like conversations with summaries. Conversations were created and written down by linguists fluent in English.

You will learn how to:

1. [Setup Development Environment](#1-setup-development-environment)
2. [Load and prepare samsum dataset](#2-load-and-prepare-samsum-dataset)
3. [Fine-tune and evaluate FLAN-T5](#3-fine-tune-and-evaluate-flan-t5)
4. [Run Inference and summarize ChatGPT dialogues](#4-run-inference-and-summarize-chatgpt-dialogues)

Before we can start, make sure you have a [Hugging Face Account](https://huggingface.co/join) to save artifacts and experiments. 

## Quick intro: FLAN-T5, just a better T5

FLAN-T5 released with the [Scaling Instruction-Finetuned Language Models](https://arxiv.org/pdf/2210.11416.pdf) paper is an enhanced version of T5 that has been finetuned in a mixture of tasks. The paper explores instruction finetuning with a particular focus on (1) scaling the number of tasks, (2) scaling the model size, and (3) finetuning on chain-of-thought data. The paper discovers that overall instruction finetuning is a general method for improving the performance and usability of pretrained language models. 

![flan-t5](flan-t5.png)

* Paper: https://arxiv.org/abs/2210.11416
* Official repo: https://github.com/google-research/t5x

## 1. Setup Development Environment

Our first step is to install the Hugging Face Libraries, including transformers and datasets. Running the following cell will install all the required packages. 

In [ ]:
import sys
!{sys.executable} -m pip install accelerate torch pytesseract transformers==4.29.0 datasets evaluate rouge-score nltk tensorboard py7zr --upgrade

## 2. Load and prepare samsum dataset

we will use the [samsum](https://arxiv.org/abs/1911.12237v2) dataset a collection of about 16k messenger-like conversations with summaries. Conversations were created and written down by linguists fluent in English.

```json
{
  "id": "13818513",
  "summary": "Amanda baked cookies and will bring Jerry some tomorrow.",
  "dialogue": "Amanda: I baked cookies. Do you want some?\r\nJerry: Sure!\r\nAmanda: I'll bring you tomorrow :-)"
}
```

It's a fairly small dataset, you can download it [from arxiv](https://arxiv.org/src/1911.12237v2/anc) or via [HuggingFace](https://huggingface.co/datasets/samsum) (which we will be doing below).

In [1]:
dataset_id = "samsum"

To load the `samsum` dataset, we use the `load_dataset()` method from the 🤗 Datasets library.


In [2]:
from datasets import load_dataset

# Load dataset from the hub
dataset = load_dataset(dataset_id)

print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

# Train dataset size: 14732
# Test dataset size: 819

Found cached dataset samsum (/Users/riedhammerko/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e)


  0%|          | 0/3 [00:00<?, ?it/s]

Train dataset size: 14732
Test dataset size: 819


Lets checkout an example of the dataset.

In [3]:
from random import randrange        


sample = dataset['train'][randrange(len(dataset["train"]))]
print(f"dialogue: \n{sample['dialogue']}\n---------------")
print(f"summary: \n{sample['summary']}\n---------------")

dialogue: 
Brenda: Hi, Jill.
Jill: Finally. Good to hear from you.
Brenda: You mean it?
Jill: Of course. I was just thinking about you.
Brenda: Any particular reason?
Jill: No. I was just thinking, it's been a long time since we've been in touch.
Brenda: Well, I thought you were mad with me.
Jill: Why would I be?
Brenda: You know. Last time we've met wasn't pure pleasure.
Jill: You mean when you met me with John.
Brenda: Yep. I had no idea, you were going out with him.
Jill: I know. And I had no idea you were his ex.
Brenda: It was awkward. And I did say too much.
Jill: That's true. But it was for the better.
Brenda: What do you mean.
Jill: Well, I took a better look at John. And we broke up.
Brenda: You did? I had no idea.
Jill: Indeed. So actually I owe you one.
Brenda: Good for you. John is a prick.
Jill: Couldn't agree more.
Brenda: So I suppose we can go for a shopping spree.
Jill: We most certainly can.
Brenda: Boy, I missed those.
Jill: Me too.
Brenda: Saturday morning then? 10 

To train our model we need to convert our inputs (text) to token IDs. This is done by a tokenizer that matches our target transformer model.

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id="google/flan-t5-small"

# Load tokenizer of FLAN-t5-small
tokenizer = AutoTokenizer.from_pretrained(model_id)


Before we can start training we need to preprocess our data. 
Abstractive summarization is a text2text-generation task where the summary is freely generated (as opposed to extractive summarization). 
This means our model will take a text as input and generate a summary as output. 
For this we want to understand how long our input and output will be to be able to efficiently batch our data. 

In [5]:
from datasets import concatenate_datasets

# The maximum total input sequence length after tokenization. 
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["dialogue"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization. 
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["summary"], truncation=True), batched=True, remove_columns=["dialogue", "summary"])
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Loading cached processed dataset at /Users/riedhammerko/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e/cache-becd8df667d1587f.arrow
Loading cached processed dataset at /Users/riedhammerko/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e/cache-64cd95cb87424ab3.arrow


Max source length: 512
Max target length: 95


In [6]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = ["summarize: " + item for item in sample["dialogue"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["dialogue", "summary", "id"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Loading cached processed dataset at /Users/riedhammerko/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e/cache-14d6d85e70c79a42.arrow
Loading cached processed dataset at /Users/riedhammerko/.cache/huggingface/datasets/samsum/samsum/0.0.0/f1d7c6b7353e6de335d444e424dc002ef70d1277109031327bc9cc6af5d3d46e/cache-ac38a595b5f790e0.arrow


Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


## 3. Fine-tune and evaluate FLAN-T5

After we have processed our dataset, we can start training our model. 
Therefore we first need to load our [FLAN-T5](https://huggingface.co/models?search=flan-t5) model from the Hugging Face Hub. 
Here we will work with the `small` version to speed up things; feel free to use a larger one (eg. `base` or `large`) and compare the performance.

In [7]:
from transformers import AutoModelForSeq2SeqLM

# huggingface hub model id
model_id="google/flan-t5-small"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

Before we get startedTo get an idea of the basic performance, let's use the original model and generate a few summaries.
To get an objective means of quality, let's use the [ROUGE metric](https://aclanthology.org/W04-1013.pdf), which basically count's an n-gram overlap between the generated and reference summary.

In [ ]:
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=0)

# select a random test sample
for i in range(5):
    sample = dataset['test'][randrange(len(dataset["test"]))]
    print(f"dialogue: \n{sample['dialogue']}\n---------------")

    # summarize dialogue
    res = summarizer(sample["dialogue"])
    print(f"reference summary:\n{sample['summary']}")
    print(f"flan-t5-small summary:\n{res[0]['summary_text']}")
    print(f"rouge metrics: {metric.compute(predictions=[res[0]['summary_text']], references=[sample['summary']], use_stemmer=True)}")

As you can see, our summaries already achieve sort of high ROUGE-1 and ROUGE-2 scores.
However: since the summaries are very short, you can already guess that this recall-oriented measure is not great-- just by getting named entities right, we can already get good scores.

We want to evaluate our model during training. 
The `Trainer` supports evaluation during training by providing a `compute_metrics`, so let's write a wrapper that decodes the tokens and compares them to the reference.

We are going to use `evaluate` library to evaluate the `rogue` score (and `punkt` to do some tokenization of the raw text).

In [8]:
import evaluate
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

# Metric
metric = evaluate.load("rouge")

# helper function to postprocess text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(sent_tokenize(label)) for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    result = {k: round(v * 100, 4) for k, v in result.items()}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    return result

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/riedhammerko/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Before we can start training is to create a `DataCollator` that will take care of padding our inputs and labels. We will use the `DataCollatorForSeq2Seq` from the 🤗 Transformers library. 

In [9]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)


The last step is to define the hyperparameters (`TrainingArguments`) we want to use for our training.

In [10]:
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# let's give the new model a nice name (flan-t5-small-samsum)
repository_id = f"{model_id.split('/')[1]}-{dataset_id}"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    predict_with_generate=True,
    fp16=False, # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=5,
    # logging & evaluation strategies
    logging_dir=f"{repository_id}/logs",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    # metric_for_best_model="overall_f1",
    # push to hub parameters
    report_to="tensorboard",
    push_to_hub=False
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    compute_metrics=compute_metrics,
)

We can start our training by using the `train` method of the `Trainer`.
Note that the training will take quite a long time (10h on an Apple M1, likely twice as much on Colab/T4).
Thus, please verify that the training runs properly, but then terminate and advance to the [next step](#4-run-inference), where you can use checkpoints and a pre-trained model provided in class.

In [11]:
# Start training
trainer.train()

/opt/homebrew/lib/python3.11/site-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/9210 [00:00<?, ?it/s]

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'loss': 1.8578, 'learning_rate': 4.728555917480999e-05, 'epoch': 0.27}
{'loss': 1.8281, 'learning_rate': 4.457111834961998e-05, 'epoch': 0.54}


Nice, we have trained our model. 🎉 Lets run evaluate the best model again on the test set.


In [ ]:
trainer.evaluate()

The best score we achieved is an `rouge1` score of `47.23`. 

In [ ]:
# Save our tokenizer and model
tokenizer.save_pretrained(repository_id)
trainer.save_model(repository_id)

## 4. Run Inference

### Optional: Load check-points or trained model

At the lab, we unfortunately don't have the hardware to run fine-tuning in decent amount of time.
Thus, your instructors will provide you with checkpoints (ie. intermediate models) and a pre-trained final model.

In [ ]:
repository_id = 'flan-t5-small-samsum'
#repository_id = 'flan-t5-small-samsum/checkpoint-XXXXX'
#repository_id = 'flan-t5-small-samsum/checkpoint-XXXXX'

tokenizer = tokenizer.from_pretrained(repository_id)
model = model.from_pretrained(repository_id)

Now we have a trained model, we can use it to run inference. We will use the `pipeline` API from transformers and a `test` example from our dataset.

In [ ]:
from transformers import pipeline
from random import randrange        

# load model and tokenizer from above
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=0)

# select a random test sample
sample = dataset['test'][randrange(len(dataset["test"]))]
print(f"dialogue: \n{sample['dialogue']}\n---------------")

# summarize dialogue
res = summarizer(sample["dialogue"])

print(f"flan-t5-small summary:\n{res[0]['summary_text']}")

# 5. Homework

1. Work with either your own or the provided checkpoints to study the effect of fine-tuning on the performance, by making a comparison of the ROUGE-1 and ROUGE-2 scores for the test set.
2. In a similar way, try out larger variants of FLAN-T5 (`base`, `large`): is it better to fine-tune a small model, or to just use a larger one?
3. Play with the fine-tuning options to see if there are any low-hanging fruit: How do learning rate and epochs affect the scores? Can you make a smaller fine-tuned model outperform a larger foundational one?